In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
# first and foremost we want data which is interprable by the network
# so we define a series of transformations to apply to our images
# to do this we define an Compose object from the torch transforms
# module which will take the images and apply 2 operations to them
# -first to convert them all to tensors
# -next normalize all the images by giving them the same mean and standard deviation
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# load the CIFAR10 dataset using our predefined transform object
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

# object to load data during the torch training process in this instance
# images are loaded in in batches of 4, and 2 subprocesses (workers)
# are used to handle loading the data
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

# this is analagous to the previous 2 steps
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [5]:
trainset[0][0].shape

torch.Size([3, 32, 32])

In [23]:
import math

import torch.nn as nn
import torch.nn.init as init

In [24]:
class VGG(nn.Module):
    '''
    VGG model 
    '''
    def __init__(self, features):
        super(VGG, self).__init__()
        self.features = features
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(512, 512),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(512, 512),
            nn.ReLU(True),
            nn.Linear(512, 10),
        )
         # Initialize weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                m.bias.data.zero_()


    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [25]:
cfg = {
    'A': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'B': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'D': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'E': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 
          512, 512, 512, 512, 'M'],
}

def make_layers(cfg, batch_norm=False):
    layers = []
    in_channels = 3
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    return nn.Sequential(*layers)

def vgg16():
    """VGG 16-layer model (configuration "D")"""
    return VGG(make_layers(cfg['D']))


def vgg16_bn():
    """VGG 16-layer model (configuration "D") with batch normalization"""
    return VGG(make_layers(cfg['D'], batch_norm=True))

In [26]:
import torch.optim as optim

In [28]:
def train(device, num_epochs=10):
    net = vgg16()
    net = net.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
    
    for epoch in range(num_epochs):  # loop over the dataset multiple times
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data[0].to(device), data[1].to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 2000 == 1999:    # print every 2000 mini-batches
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 2000))
                running_loss = 0.0
    return net

In [ ]:
device = torch.device("cuda:0")
net = train(device)

[1,  2000] loss: 2.303
[1,  4000] loss: 2.254
[1,  6000] loss: 1.998
[1,  8000] loss: 1.862
[1, 10000] loss: 1.769
[1, 12000] loss: 1.709
[2,  2000] loss: 1.616
[2,  4000] loss: 1.552
[2,  6000] loss: 1.493
[2,  8000] loss: 1.436
[2, 10000] loss: 1.363
[2, 12000] loss: 1.280
[3,  2000] loss: 1.195
[3,  4000] loss: 1.158
[3,  6000] loss: 1.087
[3,  8000] loss: 1.063
[3, 10000] loss: 1.038
[3, 12000] loss: 1.000
[4,  2000] loss: 0.885
[4,  4000] loss: 0.893
[4,  6000] loss: 0.899
[4,  8000] loss: 0.869
[4, 10000] loss: 0.845
[4, 12000] loss: 0.827
[5,  2000] loss: 0.741
[5,  4000] loss: 0.721
[5,  6000] loss: 0.728
[5,  8000] loss: 0.711
[5, 10000] loss: 0.702
[5, 12000] loss: 0.703
[6,  2000] loss: 0.604
[6,  4000] loss: 0.582
[6,  6000] loss: 0.602
[6,  8000] loss: 0.596
[6, 10000] loss: 0.599
[6, 12000] loss: 0.618
[7,  2000] loss: 0.487
[7,  4000] loss: 0.497
[7,  6000] loss: 0.502
[7,  8000] loss: 0.526
[7, 10000] loss: 0.532
[7, 12000] loss: 0.532
[8,  2000] loss: 0.408
[8,  4000] 

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))